In [86]:
import argparse
from copy import copy
import inspect
import os.path
import sys

import ipywidgets as widgets

from symdesign import flags, utils
from symdesign.SymDesign import app

In [6]:
# %load_ext shell
!echo $PYTHONPATH

PYTHONPATH: Undefined variable.


In [2]:
os.environ['PYTHONPATH']

'/home/kmeador/symdesign'

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
def get_class_that_defined_method(meth):
    """Where meth is a bound method"""
    if inspect.ismethod(meth):
        for cls in inspect.getmro(meth.__self__.__class__):
            if meth.__name__ in cls.__dict__:
                return cls
    return None


# All modules
# flags.available_modules
notebook_allowed_modules = [
    flags.align_helices,
    flags.analysis,
    flags.check_clashes,
    flags.cluster_poses,
    flags.design,
    flags.expand_asu,
    flags.generate_fragments,
    flags.helix_bending,
    flags.protocol,
    flags.rename_chains,
    flags.select_designs,
    flags.select_poses,
    flags.select_designs,
]
module_help = widgets.Label(value='Please input the module(s) you would like to include with this job')
module_tags = widgets.TagsInput(
    value=[flags.align_helices],
    allowed_tags=notebook_allowed_modules,
    allow_duplicates=False
)
module_input = widgets.VBox([module_help, module_tags])
display(module_input)

In [82]:
choices_widget = widgets.Dropdown
multiple_text = widgets.TagsInput
single_text = widgets.Text
boolean_widget = widgets.Checkbox  # widgets.ToggleButton

def process_module_arg_to_widget(arg, widget_kwargs):
    if arg.nargs:
        widget = multiple_text(**widget_kwargs)
    else:
        if arg.metavar:
            placeholder = arg.metavar
        else:
            placeholder = ''  # arg.default
        widget = single_text(**widget_kwargs, placeholder=placeholder)

    return widget

# Use with [options-inputs row, ...] formating
row_layout = widgets.Layout(display='flex',
                            flex_flow='row', 
                            # justify_content='space-between',
                            align_content='stretch',
                            align_items='stretch'
                            # border='solid',
                            # width='100%'
                           )
box_layout = widgets.Layout(display='flex',
                            flex_flow='column', 
                            align_items='stretch', 
                            border='solid',
                            width='100%')
# # Use with options column / inputs column formating
# row_layout = widgets.Layout(display='flex',
#                             flex_flow='column', 
#                             justify_content='center',
#                             align_content='stretch',
#                             align_items='stretch'
#                             # border='solid',
#                             # width='100%'
#                            )
# description_row_layout = widgets.Layout(display='flex',
#                             flex_flow='column', 
#                             justify_content='center',
#                             align_content='flex-end',
#                             align_items='flex-end',
#                             # border='solid',
#                             width='100%'
#                            )
# box_layout = widgets.Layout(display='flex',
#                             flex_flow='row', 
#                             align_items='stretch', 
#                             border='solid',
#                             width='100%')
description_layout = widgets.Layout(display='flex',
                                    justify_content='flex-end',
                                    width='25%')
module_use_description = widgets.Label(value='Fill out a form for each module to enable various job options')
module_help = widgets.HTML(value='<b>Module description:</b>')
required_widget = widgets.Valid  # (value=False, description=None)

# Add each requested as its own widget, formatting module arguments
all_module_widgets = {}  # widgets.IntSlider(), widgets.Text()]
# for group in parser._action_groups:
for group in flags.argparsers[flags.parser_entire]._action_groups:
    for arg in group._group_actions:
        if isinstance(arg, argparse._SubParsersAction):
            for module, subparser in arg.choices.items():
                # if module in module_tags.value:
                try:
                    module_index = module_tags.value.index(module)
                except ValueError:
                    continue
                else:
                    
                    # module_description = widgets.Label(value=subparser.description)
                    module_description = widgets.HTML(value=subparser.description)
                    
                    module_widgets = [module_help, module_description]
                    # # Use with options column / inputs column formating
                    # module_widgets = [module_description]
                    # description_widgets = [module_help]
                    for group in subparser._action_groups:
                        # These are the processed module arguments
                        for module_arg in group._group_actions:
                            # description = f'--{module_arg.dest}'
                            # print(module_arg)
                            # help = module_arg.help
                            choices = module_arg.choices
                            required = module_arg.required
                            if required:
                                print('Required', module_arg.option_strings[-1])
                            type_ = module_arg.type
                            
                            if module_arg.default is None:
                                if module_arg.nargs:
                                    print(module_arg.option_strings[-1], 'is None, but has nargs')
                                    module_value = None  # [] # None]
                                    print(module_arg)
                                    # choices = none_tuple + copy(choices)
                                else:
                                #     format_widget = lambda widget: widget
                                    module_value = module_arg.default
                            elif isinstance(module_arg.default, bool):
                                module_value = module_arg.default
                            elif isinstance(module_arg.default, (int, float)):
                                module_value = str(module_arg.default)
                            else:
                                module_value = module_arg.default
                            
                            description = module_arg.option_strings[-1]
                            tooltip = module_arg.help % vars(module_arg)
                            widget_kwargs = dict(value=module_value,
                                                 # description=module_arg.option_strings[-1],
                                                 tooltip=tooltip)
                            # except TypeError as error:
                                # print(error)
                                # input(module_arg.help)
                            if isinstance(module_arg, (argparse._StoreTrueAction, argparse._StoreFalseAction,
                                                       argparse.BooleanOptionalAction)):  # .action in boolean_actions:
                                if isinstance(module_arg, argparse.BooleanOptionalAction):
                                    # Swap the "--no-" prefixed flag for the typical prefix
                                    # widget_kwargs.pop('description')
                                    # widget_kwargs['description'] = module_arg.option_strings[-2]
                                    description = module_arg.option_strings[-2]
                                    
                                # widget = boolean_widget(description=description, **widget_kwargs)  # , tooltip=help)
                                # Using the Checkbox
                                widget = boolean_widget(indent=False, **widget_kwargs)
                                # widget = widgets.Box([widgets.Label(value=description, tooltip=tooltip, layout=description_layout),
                                #                       widget], layout=row_layout)
                            elif isinstance(module_arg, argparse._StoreAction):
                                if choices:
                                    if module_arg.default not in choices:
                                        # When there is no default provided
                                        choices_ = (module_arg.default, *choices)
                                    else:
                                        choices_ = choices
                                    print(choices_)
                                    widget = choices_widget(**widget_kwargs, options=choices_)
                                    print(widget)
                                elif type_ is None or get_class_that_defined_method(type_) == str:
                                    # These are processed as strings
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # if module_arg.nargs:
                                    #     widget = multiple_text(**widget_kwargs)
                                    # else:
                                    #     if module_arg.metavar:
                                    #         placeholder = module_arg.metavar
                                    #     else:
                                    #         placeholder = ''
                                    #     widget = single_text(**widget_kwargs, placeholder=placeholder)
                                elif type_ == int:
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # if module_arg.nargs:
                                    #     # There are currently none of these
                                    #     widget = widgets.IntsInput(**widget_kwargs)
                                    # else:
                                    #     widget = widgets.IntText(**widget_kwargs)
                                elif type_ == float:
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # if module_arg.nargs:
                                    #     # There are currently none of these
                                    #     widget = widgets.FloatsInput(**widget_kwargs)
                                    # else:
                                    #     widget = widgets.FloatText(**widget_kwargs)
                                    # widget = widgets.HBox([widget, widgets.Label(value=module_arg.metavar)])
                                elif type_ == os.path.abspath:
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # if module_arg.nargs:
                                    #     # There are currently none of these
                                    #     widget = multiple_text(**widget_kwargs, placeholder=module_arg.metavar)
                                    # else:
                                    #     if module_arg.metavar:
                                    #         placeholder = module_arg.metavar
                                    #     else:
                                    #         placeholder = ''
                                    #     widget = single_text(**widget_kwargs, placeholder=placeholder)
                                # Custom types
                                elif type_ == flags.temp_gt0:
                                    print(widget_kwargs['value'])
                                    widget_kwargs['value'] = [str(val) for val in widget_kwargs['value']]
                                    print(widget_kwargs['value'])
                                    widget = process_module_arg_to_widget(module_arg, widget_kwargs)
                                    # widget = widgets.FloatsInput(**widget_kwargs)
                                
                                # widgets.HBox([widget, widgets.Label(value=module_arg.help)])
                                # widget = widgets.HBox([widgets.Label(value=widget_kwargs['description']), widget])
                                # widget = widgets.HBox([widgets.Label(value=description), widget])
                                # widget = widgets.Box([widgets.Label(value=description, tooltip=tooltip, layout=description_layout),
                                #                       widget], layout=row_layout)
                            else:
                                continue

                            # DEBUG
                            # if 'pdb-code' in module_arg.option_strings[-1]:
                            #     print(module_arg)
                            #     print(module_arg.option_strings[-1])
                            #     print(widget_kwargs)
                            #     print(widget)
                            #     display(widget)
                            desc_widget = widgets.Label(value=description, tooltip=tooltip, layout=description_layout)
                            # Add the module widgets to all widgets
                            # Use with [options-inputs row, ...] formating
                            widget = widgets.Box([desc_widget, widget], layout=row_layout)
                            module_widgets.append(widget)
                            # # Use with options column / inputs column formating
                            # description_widgets.append(desc_widget)
                            # module_widgets.append(widget)
                    
                    # Used for descriptions in each widget rather than an HBox description
                    # max_description_length = 0
                    # for widget in module_widgets:
                    #     if len(widget.description) > max_description_length:
                    #         max_description_length = len(widget.description)
                    #         print(max_description_length)
                    #         print(widget.style)
                    # module_layout = {'description_width': f'{max_description_length * 6}px'}
                    # print(max_description_length * 5)
                    # for widget in module_widgets:
                    #     widget.style = module_layout

                    module_widget = widgets.Box(children=module_widgets, layout=box_layout)
                    # # Use with options column / inputs column formating
                    # description_column = widgets.Box(children=description_widgets, layout=description_row_layout)
                    # module_column = widgets.Box(children=module_widgets, layout=row_layout)
                    # module_widget = widgets.Box(children=[description_column, module_column])  # , layout=box_layout)

                    # Add the module widgets to all widgets
                    # all_module_widgets.append(widgets.VBox(module_widgets))
                    # all_module_widgets.append(module_widget)
                    all_module_widgets[module_index] = module_widget

                    
# For formatting the help, use
# widgets.HBox([WIDGET, widgets.Label(value="The $m$ in $E=mc^2$:")])
# for boolean flags, use
# widgets.ToggleButton(
#     value=False,
#     description='Click me',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Description',
#     icon='check' # (FontAwesome names without the `fa-` prefix)
# )
# For choices flags, use
# widgets.Dropdown(
#     options=[('One', 1), ('Two', 2), ('Three', 3)],
#     # options=['1', '2', '3'],
#     value=2,
#     description='Number:',
# )
# For int flags, use
# widgets.IntText(
#     value=7,
#     description='Any:',
#     disabled=False
# )
# widgets.IntsInput(
#     value=[1, 4, 3243],
#     min=0,
#     max=1000000,
#     format='$,d'
# )
# For float flags, use
# widgets.FloatText(
#     value=7,
#     description='Any:',
#     disabled=False
# )
# For string flags, use
# widgets.Text(
#     value='Hello World',
#     placeholder='Type something',
#     description='String:',
#     disabled=False
# )

# Set up the display to show all options for each module
# # Accordian display
# options_accordion = widgets.Accordion(children=all_module_widgets, titles=module_tags.value)
# display(options_accordion)
# Tab display
module_options_tab = widgets.Tab()
module_options_tab.children = [all_module_widgets[idx] for idx in range(len(module_tags.value))]
module_options_tab.titles = module_tags.value  # ['align', 'design']
module_box_layout = widgets.Layout(display='flex',
                            flex_flow='column', 
                            align_items='stretch', 
                            border='solid',
                            width='50%')
module_box = widgets.Box([module_use_description, module_options_tab], layout=module_box_layout)  # widgets.Layout(display='flex', flex_flow='column'))
display(module_box)

--target-termini is None, but has nargs
_StoreAction(option_strings=['--target-termini'], dest='target_termini', nargs='*', const=None, default=None, type=<method 'lower' of 'str' objects>, choices=('n', 'c'), required=False, help="If particular termini are desired, specify with 'n' and/or 'c'", metavar=None)
(None, 'n', 'c')
Dropdown(options=(None, 'n', 'c'), tooltip="If particular termini are desired, specify with 'n' and/or 'c'", value=None)
('consensus', 'proteinmpnn', 'rosetta')
Dropdown(index=1, options=('consensus', 'proteinmpnn', 'rosetta'), tooltip="Which design method should be used?\nChoices=('consensus', 'proteinmpnn', 'rosetta')\nDefault=proteinmpnn", value='proteinmpnn')
['v_48_002', 'v_48_010', 'v_48_020', 'v_48_030']
Dropdown(index=2, options=('v_48_002', 'v_48_010', 'v_48_020', 'v_48_030'), tooltip="The name of the model to use for proteinmpnn design/scoring\nwhere the model name takes the form v_X_Y, with X indicating\nThe number of neighbors, and Y indicating the tra

Box(children=(Label(value='Fill out a form for each module to enable various job options'), Tab(children=(Box(…

In [58]:
out = widgets.Output(layout={'border': '1px solid black'})

In [91]:
def parse_user_input():
    """Format the arguments input from user into parsable command-line like arguments"""
    if len(module_tags.value) > 1:
        valid_arguments = [['symdesign', 'protocol', '--modules', *module_tags.value]]
    else:
        valid_arguments = ['symdesign', *module_tags.value]

    for module_widget_flag_argument in module_options_tab.children:
        for flag_argument in module_widget_flag_argument.children:
            if getattr(flag_argument, 'children', None):
                flag, argument = flag_argument.children
                if argument.value:
                    if isinstance(argument.value, bool):
                        argument = [flag.value]
                    elif isinstance(argument.value, list):
                        argument = [flag.value, *argument.value]
                    else:
                        argument = [flag.value, argument.value]
                    valid_arguments.append(argument)
                    # print(valid_arguments[-1])

    sys.argv = []
    for arg in valid_arguments:
        # sys.argv = valid_arguments
        sys.argv.extend(arg)

    # # Debugging
    # sys.argv.append('--help')
    # print(f'Running with the user arguments:\n{sys.argv}')

@out.capture(clear_output=True)
def run_app(*args):
    parse_user_input()
    try:
        app()
    except SystemExit:
        return
    except utils.SymDesignException as error:
        print(error)

In [94]:
run_protocol_button = widgets.Button(
    description='Run protocol',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='To perform the protocols specified, click this button',
    icon='turn-down-left'
)
run_protocol_button.on_click(run_app)
display(run_protocol_button)

Button(button_style='success', description='Run protocol', icon='turn-down-left', style=ButtonStyle(), tooltip…

In [60]:
out

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [31]:
from collections import defaultdict
from IPython.display import HTML
import ipywidgets
from pprint import pprint

In [32]:
reverse_lut = defaultdict(set)
styles = set()
for export_name in dir(ipywidgets.widgets):
    export = getattr(ipywidgets.widgets, export_name)
    try:
        if issubclass(export, ipywidgets.Widget) and 'style' in export.class_trait_names():
            reverse_lut[export.style.klass.__name__].add(export.__name__)
            styles.add(export.style.klass)
    except TypeError:
        pass

html = '<ul>'
for style, widgets_ in reverse_lut.items():
    html = f"{html}\n<li><b>{style}:</b> {', '.join(sorted(widgets_))}</li>"
html += "</ul>"
HTML(html)

In [62]:
attributes = defaultdict(set)
base_traits = set(ipywidgets.Style.class_trait_names())

for s in styles:
    for t in s.class_trait_names():
        if not t.startswith("_") and t not in base_traits:
            attributes[s.__name__].add(t)
all_attributes = set().union(*attributes.values())

html = '<table>\n'
html = f"{html}<tr><th>Attribute</th>{ ''.join(f'<th>{s}</th>' for s in attributes.keys()) }</tr>"
for a in all_attributes:
    html = f"""{html}<tr><td>{a}</td>{ ''.join(f'<td>{"✓" if a in attribs else ""}</td>' for attribs in attributes.values()) }</tr>"""
html += "</table>"
HTML(html)

Attribute,LabelStyle,SliderStyle,CheckboxStyle,DescriptionStyle,HTMLStyle,ToggleButtonStyle,ToggleButtonsStyle,TextStyle,HTMLMathStyle,ProgressStyle,ButtonStyle
background,✓,,✓,,✓,,,✓,✓,,
font_weight,✓,,,,,✓,✓,,,,✓
handle_color,,✓,,,,,,,,,
description_width,✓,✓,✓,✓,✓,✓,✓,✓,✓,✓,
font_variant,✓,,,,,✓,,,,,✓
button_width,,,,,,,✓,,,,
text_decoration,✓,,,,,✓,,,,,✓
font_family,✓,,,,,✓,,,,,✓
font_size,✓,,,,✓,✓,,✓,✓,,✓
font_style,✓,,,,,✓,,,,,✓
